In [ ]:
%%capture
# !pip install llama-index==0.10.37 cohere==5.5.0 openai==1.30.1 llama-index-embeddings-openai==0.1.9 llama-index-llms-cohere==0.2.0 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 

In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [ ]:
# OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [4]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [6]:
from pathlib import Path

def create_directory(directory_name):
    path = Path(directory_name)
    path.mkdir(parents=True, exist_ok=True)
    print(f"Directory '{directory_name}' created successfully.")

create_directory("data")

Directory 'data' created successfully.


In [7]:
!wget -P data https://www.gutenberg.org/cache/epub/10763/pg10763.txt 

--2025-07-24 16:56:35--  https://www.gutenberg.org/cache/epub/10763/pg10763.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405150 (396K) [text/plain]
Saving to: ‘data/pg10763.txt’

pg10763.txt         100%[===================>] 395.65K  2.00MB/s    in 0.2s    

2025-07-24 16:56:35 (2.00 MB/s) - ‘data/pg10763.txt’ saved [405150/405150]



# 🗄️ Storing

Loading and indexing data costs time and money.

By default, indexed data is stored in memory. But, you can store your data to avoid the time and costs associated with re-indexing them.  The simplest way to do this **persisting to disk**.

Each `Index` object has a `.persist()` method, which will write all the data to disk at the specified location.

Now that we've dowloaded data, let's:

1) Load as Document
2) Parse as Nodes
3) Create index

In [8]:
from llama_index.core import SimpleDirectoryReader

file_path = "data/pg10763.txt"

document = SimpleDirectoryReader(input_files=[file_path], filename_as_id=True).load_data()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/lil_llama_index/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
# Create Node parser
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(
    chunk_size=512, 
    chunk_overlap=16,
    paragraph_separator="\n\n\n\n" 
)

In [ ]:
# Instantiate embedding model
from llama_index.embeddings.openai import OpenAIEmbedding

# embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [15]:
from llama_index.embeddings.cohere import CohereEmbedding

embed_model = CohereEmbedding(model_name="embed-english-v3.0")

# ☁️ Using a Vector Database

We'll use qdrant as our vector database of choice throughout this course.

To use qdrant to store embeddings from the `VectorStoreIndex`, you need to:

- Initialize the qdrant client

- Create a `Collection` to store your data in qdrant

- Assign qdrant as the `vector_store` in a `StorageContext`

- Initialize your `VectorStoreIndex` using that `StorageContext`

Below, we initialize a `QdrantClient` for interacting with qdrant, an open-source vector store. 


In [16]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# initialize qdrant client
client = qdrant_client.QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

vector_store = QdrantVectorStore(
    client=client, 
    collection_name="it_can_be_done",
    embed_model=embed_model,
)

# 🗃️ Storage Context

`StorageContext` in `LlamaIndex` is a core abstraction that revolves around the storage of `Nodes`, indices, and vectors.  It facilitates data storage and retrieval.

It is a utility container that supports the following:

 - `docstore`: A [`BaseDocumentStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/docstore/types.py) for storing nodes.

 - `index_store`: A [`BaseIndexStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/index_store/types.py#L13) for storing indices.

 - `vector_store`: A [`VectorStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/vector_stores/simple.py) for storing vectors.

 - `graph_store`: A [`GraphStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/graph_stores/simple.py) for storing knowledge graphs.

Below we instantiate the `StorageContext` from default settings indicating that we want to use a vector store.

In [17]:
from llama_index.core import StorageContext

# assign qdrant vector store to storage context
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

In [18]:
from llama_index.core import  VectorStoreIndex

# create the index
index = VectorStoreIndex.from_documents(
    document,
    show_progress=True,
    store_nodes_override=True,
    transformation=[sentence_splitter],
    embed_model=embed_model,
    storage_context=storage_context,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/126 [00:00<?, ?it/s]

# 🪃 Retrieval

A `Retriever` is an interface exposed by the `Index`. An `Index` with its `Retriever` is used for storing and fetching data. The `Retriever` is a part of the `Index` and is used to retrieve the data stored in the Index.


### LlamaIndex provides [many different types of retrievers](https://github.com/run-llama/llama_index/tree/main/llama-index-core/llama_index/core/retrievers) to fetch relevant information from ingested data based on a given query. 

Some examples include

### Vector Retriever

The vector retriever uses vector similarity search to find the most relevant nodes (chunks of text) based on the query embedding. It requires a vector database like to store and search through the node embeddings.

### [Fusion Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/fusion_retriever.py)

The fusion retriever generates multiple queries from the original query, performs retrieval over an ensemble of retrievers for each query, and then fuses and reranks the results across all queries. This aims to better capture the query intent through query rewriting and ensembling.

### [Recursive Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/recursive_retriever.py)

The recursive retriever allows for hierarchical retrieval by first retrieving coarse nodes and then recursively retrieving finer-grained nodes within those coarse nodes. This can be useful for multi-level indexing and retrieval.

You can also combine retrievers in interesting ways and build out more advanced retrieval strategies, as we will see later in this course.


### In the example here, we're using a Vector Retriever

 - 🔍 When searching, your query is also converted into a vector embedding. 
 
- 🗂️ The `VectorStoreIndex` then performs a mathematical operation to rank embeddings based on semantic similarity to your query.

- 🔝 Top-k semantic retrieval is the simplest wasy to query a vector index.

- ⩬ You can also apply a similarity threshold  (e.g., only return results that are more similar than some value)


In [19]:
retirever = index.as_retriever(
    similarity_top_k=5,
    similarity_threshold=0.75)

In [20]:
retirever.retrieve("What lessons can be learned from the poems about success?")

[NodeWithScore(node=TextNode(id_='b135c767-9bbf-4f3f-8f91-e6f89d88faa4', embedding=None, metadata={'file_path': 'data/pg10763.txt', 'file_name': 'pg10763.txt', 'file_type': 'text/plain', 'file_size': 405150, 'creation_date': '2025-07-24', 'last_modified_date': '2025-07-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='data/pg10763.txt', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'data/pg10763.txt', 'file_name': 'pg10763.txt', 'file_type': 'text/plain', 'file_size': 405150, 'creation_date': '2025-07-24', 'last_modified_date': '2025-07-05'}, hash='2e22f099825e416e7974219ee0934129b343fb779ba9c8e30930f7324644ed75'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='b6fc5767-dff7-4

But, chances are you don't just want the returned documents. You want the documents to be synthesized into a response. 

So, let's build on this pattern in the next lesson and see how we can get a response based on those retrieved documents.

In [21]:
# close the client so you're not locked out of the index
client.close()